<a href="https://colab.research.google.com/github/rmahesh6386/DataScience_Python/blob/main/Assignment_11_Random__Forest_Company.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
company = pd.read_csv("Company_Data.csv")
company.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No


In [22]:
company.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No


In [23]:
#first convert sales into categorical variable
company['Sales_Rating'] = pd.cut(company.Sales,bins=[0,3,6,10,14,20],
                                      labels=['Bad','Below_Avg','Avg','Good','Excellent'],include_lowest=True)

In [24]:
from sklearn.preprocessing import LabelEncoder
label= LabelEncoder()
company1 = company[['ShelveLoc', 'Urban','US']].apply(label.fit_transform)
company1 = company1.rename(columns={'ShelveLoc':'ShelveLoc_num','Urban':'Urban_num','US':'US_num'})
company1.head()

,ShelveLoc_num,Urban_num,US_num
0,0,1,1
1,1,1,1
2,2,1,1
3,2,1,1
4,0,1,0


In [25]:
company = pd.concat([company,company1], axis=1, sort=False)
company.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,Sales_Rating,ShelveLoc_num,Urban_num,US_num
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes,Avg,0,1,1
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes,Good,1,1,1
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes,Good,2,1,1
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes,Avg,2,1,1
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No,Below_Avg,0,1,0


In [26]:
company =company.drop(columns={'Sales','ShelveLoc','Urban','US'},axis=1)

company = company[["CompPrice","Income","Advertising","Population","Price","Age",
                             "Education","Urban_num","US_num","ShelveLoc_num","Sales_Rating"]]


company.head()

,CompPrice,Income,Advertising,Population,Price,Age,Education,Urban_num,US_num,ShelveLoc_num,Sales_Rating
0,138,73,11,276,120,42,17,1,1,0,Avg
1,111,48,16,260,83,65,10,1,1,1,Good
2,113,35,10,269,80,59,12,1,1,2,Good
3,117,100,4,466,97,55,14,1,1,2,Avg
4,141,64,3,340,128,38,13,1,0,0,Below_Avg


In [28]:
colnames=list(company.columns)
colnames

['CompPrice',
 'Income',
 'Advertising',
 'Population',
 'Price',
 'Age',
 'Education',
 'Urban_num',
 'US_num',
 'ShelveLoc_num',
 'Sales_Rating']

In [29]:
x=company.iloc[:,0:10]
y=company['Sales_Rating']

In [30]:
x.head()

,CompPrice,Income,Advertising,Population,Price,Age,Education,Urban_num,US_num,ShelveLoc_num
0,138,73,11,276,120,42,17,1,1,0
1,111,48,16,260,83,65,10,1,1,1
2,113,35,10,269,80,59,12,1,1,2
3,117,100,4,466,97,55,14,1,1,2
4,141,64,3,340,128,38,13,1,0,0


In [38]:
y

0            Avg
1           Good
2           Good
3            Avg
4      Below_Avg
         ...    
395         Good
396          Avg
397          Avg
398    Below_Avg
399          Avg
Name: Sales_Rating, Length: 400, dtype: category
Categories (5, object): ['Bad' < 'Below_Avg' < 'Avg' < 'Good' < 'Excellent']

In [34]:
company.isnull().any()

CompPrice        False
Income           False
Advertising      False
Population       False
Price            False
Age              False
Education        False
Urban_num        False
US_num           False
ShelveLoc_num    False
Sales_Rating     False
dtype: bool

In [35]:
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

kfold = KFold(n_splits=10)
model = RandomForestClassifier(n_estimators=20, max_features=3)
results = cross_val_score(model, x, y, cv=kfold)
print(results.mean())

0.55


In [36]:
model.fit(x,y)
company['pred'] = model.predict(x)
cols = ['pred','Sales_Rating']
company[cols].head()

,pred,Sales_Rating
0,Avg,Avg
1,Good,Good
2,Good,Good
3,Avg,Avg
4,Below_Avg,Below_Avg


In [37]:
from sklearn.metrics import confusion_matrix
confusion_matrix(company["Sales_Rating"],model.predict(x)) # Confusion matrix

pd.crosstab(company["Sales_Rating"],company["pred"])

pred,Avg,Bad,Below_Avg,Excellent,Good
Sales_Rating,,,,,
Bad,0,17,0,0,0
Below_Avg,0,0,113,0,0
Avg,192,0,0,0,0
Good,0,0,0,0,74
Excellent,0,0,0,4,0


In [40]:
print("Accuracy",(74+192+113+17+4)/(74+192+113+17+4)*100)

Accuracy 100.0
